# Hydrodynamic Coefficients Analysis Example

This notebook demonstrates the complete functionality of the hydrodynamic coefficients module including:
- Loading coefficient data from CSV files
- Interpolating coefficients at arbitrary frequencies
- Validating data quality and causality
- Creating comprehensive visualizations
- Exporting results

## 1. Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Import hydrodynamic coefficients module
from marine_engineering.hydrodynamic_coefficients import (
    CoefficientDatabase,
    HydrodynamicPlotter,
    DOF_NAMES,
    AQWAParser
)

# Configure display
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

print("Imports successful!")
print(f"Available DOFs: {DOF_NAMES}")

## 2. Load Hydrodynamic Coefficient Database

In [ ]:
# Path to hydrodynamic coefficient CSV files
data_dir = Path('../data/marine_engineering/hydrodynamic/')

# Load database
db = CoefficientDatabase.from_csv(data_dir)

print(f"Database loaded: {db}")
print(f"Number of frequencies: {len(db.frequencies)}")
print(f"Frequency range: {db.get_frequency_range()} rad/s")

# Display available frequencies
print("\nAvailable frequencies (rad/s):")
print(db.frequencies[:10], "...")  # First 10

## 3. Explore Coefficient Data

In [ ]:
# Get coefficient at specific frequency and DOF
frequency = 0.5  # rad/s
A33 = db.get_added_mass(frequency, dof_i='Heave', dof_j='Heave')
B33 = db.get_damping(frequency, dof_i='Heave', dof_j='Heave')

print(f"At frequency {frequency} rad/s:")
print(f"  Added Mass (Heave-Heave): {A33:.2f}")
print(f"  Damping (Heave-Heave): {B33:.2f}")

# Get full matrices at frequency
A_matrix = db.get_added_mass_matrix(frequency)
B_matrix = db.get_damping_matrix(frequency)

print(f"\nAdded Mass Matrix shape: {A_matrix.shape}")
print(f"Damping Matrix shape: {B_matrix.shape}")

## 4. Statistical Analysis

In [ ]:
# Get statistics for added mass coefficients
stats_am = db.get_coefficient_statistics('added_mass')
print("Added Mass Statistics:")
print(stats_am.head(10))

# Get statistics for damping coefficients
stats_damp = db.get_coefficient_statistics('damping')
print("\nDamping Statistics:")
print(stats_damp.head(10))

## 5. Causality Validation (Kramers-Kronig)

In [ ]:
# Validate causality for heave DOF
is_valid, error = db.validate_causality('Heave', 'Heave', tolerance=0.2)

print(f"Kramers-Kronig Validation for Heave-Heave:")
print(f"  Valid: {is_valid}")
print(f"  Maximum Error: {error:.4f}")

# Validate all diagonal terms
print("\nValidation for all DOFs:")
for dof in DOF_NAMES:
    is_valid, error = db.validate_causality(dof, dof, tolerance=0.2)
    status = "✓" if is_valid else "✗"
    print(f"  {status} {dof}: error = {error:.4f}")

## 6. Visualization - Frequency Response

In [ ]:
# Create plotter
plotter = HydrodynamicPlotter(db)

# Plot frequency response for Heave DOF
fig = plotter.plot_frequency_response(
    dof='Heave',
    coefficient_type='both',
    coupled=False,
    save_path='../outputs/hydrodynamic_charts/heave_frequency_response.png',
    figsize=(14, 6)
)
plt.show()

print("Heave frequency response plotted.")

## 7. Visualization - Coefficient Matrix Heatmap

In [ ]:
# Plot added mass matrix at specific frequency
fig = plotter.plot_coefficient_matrix(
    frequency=0.5,
    coefficient_type='added_mass',
    save_path='../outputs/hydrodynamic_charts/added_mass_matrix_heatmap.png'
)
plt.show()

# Plot damping matrix at specific frequency
fig = plotter.plot_coefficient_matrix(
    frequency=0.5,
    coefficient_type='damping',
    save_path='../outputs/hydrodynamic_charts/damping_matrix_heatmap.png'
)
plt.show()

print("Matrix heatmaps created.")

## 8. Visualization - 3D Surfaces

In [ ]:
# Plot 3D surface for Heave added mass
fig = plotter.plot_added_mass_surface(
    dof_i='Heave',
    dof_j='Heave',
    save_path='../outputs/hydrodynamic_charts/heave_added_mass_surface.png'
)
plt.show()

# Plot 3D surface for Heave damping
fig = plotter.plot_damping_surface(
    dof_i='Heave',
    dof_j='Heave',
    save_path='../outputs/hydrodynamic_charts/heave_damping_surface.png'
)
plt.show()

print("3D surfaces created.")

## 9. Visualization - Cross-Coupling Terms

In [ ]:
# Plot cross-coupling for added mass
fig = plotter.plot_cross_coupling(
    coefficient_type='added_mass',
    threshold=50.0,
    save_path='../outputs/hydrodynamic_charts/added_mass_cross_coupling.png'
)
plt.show()

# Plot cross-coupling for damping
fig = plotter.plot_cross_coupling(
    coefficient_type='damping',
    threshold=0.1,
    save_path='../outputs/hydrodynamic_charts/damping_cross_coupling.png'
)
plt.show()

print("Cross-coupling plots created.")

## 10. Visualization - Critical Damping Ratios

In [ ]:
# Define system properties (example values)
mass_matrix = np.diag([10000, 10000, 10000, 1000, 1000, 1000])
stiffness_matrix = np.diag([50000, 50000, 50000, 5000, 5000, 5000])

# Plot critical damping ratios
fig = plotter.plot_critical_damping(
    mass_matrix=mass_matrix,
    stiffness_matrix=stiffness_matrix,
    save_path='../outputs/hydrodynamic_charts/critical_damping_ratios.png'
)
plt.show()

print("Critical damping ratios plotted.")

## 11. Visualization - Coefficient Correlation

In [ ]:
# Plot correlation matrix for added mass
fig = plotter.plot_matrix_correlation(
    coefficient_type='added_mass',
    save_path='../outputs/hydrodynamic_charts/added_mass_correlation.png'
)
plt.show()

print("Correlation matrix created.")

## 12. Visualization - All DOFs Comparison

In [ ]:
# Compare all DOFs for added mass
fig = plotter.plot_all_dofs_comparison(
    coefficient_type='added_mass',
    save_path='../outputs/hydrodynamic_charts/all_dofs_added_mass.png'
)
plt.show()

# Compare all DOFs for damping
fig = plotter.plot_all_dofs_comparison(
    coefficient_type='damping',
    save_path='../outputs/hydrodynamic_charts/all_dofs_damping.png'
)
plt.show()

print("All DOFs comparison created.")

## 13. Create Animated Frequency Sweep

In [ ]:
# Create animation for added mass matrix evolution
plotter.animate_frequency_sweep(
    coefficient_type='added_mass',
    save_path='../outputs/hydrodynamic_charts/added_mass_animation.gif',
    fps=5
)

print("Animation created: added_mass_animation.gif")

## 14. Export Results

In [ ]:
# Export coefficient data to new directory
export_dir = Path('../outputs/exported_coefficients/')
db.export_to_csv(export_dir)

print(f"Coefficients exported to: {export_dir}")

# Export statistics to Excel
stats_am = db.get_coefficient_statistics('added_mass')
stats_damp = db.get_coefficient_statistics('damping')

with pd.ExcelWriter('../outputs/coefficient_statistics.xlsx') as writer:
    stats_am.to_excel(writer, sheet_name='Added Mass', index=False)
    stats_damp.to_excel(writer, sheet_name='Damping', index=False)

print("Statistics exported to: coefficient_statistics.xlsx")

## 15. Advanced Example: Frequency Response Analysis

In [ ]:
# Analyze frequency response for specific DOF with coupling
fig = plotter.plot_frequency_response(
    dof='Pitch',
    coefficient_type='both',
    coupled=True,  # Show all coupling terms
    save_path='../outputs/hydrodynamic_charts/pitch_coupled_response.png',
    figsize=(16, 7)
)
plt.show()

print("Coupled frequency response created.")

## 16. Summary Report

In [ ]:
# Generate summary report
freq_min, freq_max = db.get_frequency_range()

print("=" * 60)
print("HYDRODYNAMIC COEFFICIENTS ANALYSIS SUMMARY")
print("=" * 60)
print(f"\nDatabase Information:")
print(f"  Number of frequencies: {len(db.frequencies)}")
print(f"  Frequency range: {freq_min:.3f} - {freq_max:.3f} rad/s")
print(f"  Period range: {2*np.pi/freq_max:.2f} - {2*np.pi/freq_min:.2f} s")

print(f"\nAvailable Matrices:")
print(f"  Added Mass matrices: {len(db.added_mass_matrices)}")
print(f"  Damping matrices: {len(db.damping_matrices)}")

print(f"\nGenerated Visualizations:")
print(f"  ✓ Frequency response plots")
print(f"  ✓ Matrix heatmaps")
print(f"  ✓ 3D surface plots")
print(f"  ✓ Cross-coupling analysis")
print(f"  ✓ Critical damping ratios")
print(f"  ✓ Correlation matrices")
print(f"  ✓ All DOFs comparison")
print(f"  ✓ Animated frequency sweep")

print(f"\nOutput Directory: ../outputs/hydrodynamic_charts/")
print("=" * 60)